#  Integration

Solve the following exercises.

1. Prove that the midpoint rule is exact for polynomials of degree $\leq 1$.

2. Compute the coefficients for the closed Newton-Cotes rule with four points.

And also do exercises 13.7 and 13.20 from the book.

Let us now implement numerical integration. Ideally, we would like to write something of the form

``function integrate(f::Function, a::Real, b::Real)
...
end
``

that would return an approximate value for 
$$ \int_a^b f(x) \, dx $$

We will write different versions of numerical quadrature. Let us start with the trapezoid rule using $n+1$ points (from $x_0$ to $x_n$).

In [1]:
function trapezoid(f::Function, a::Real, b::Real, n::Integer=100)
    sum = 0.
    h = (b-a)/n
    sum += (f(a)+f(b))*h/2
    for i in 1:n-1
        xi = h*i + a
        sum += h*f(xi)
    end
    return sum
end

trapezoid (generic function with 2 methods)

Let us verify that 
$$ \int_0^1 x^2 dx = 1/3 $$.

In [2]:
trapezoid(x->x^2,0,1)

0.33335000000000015

We now implement the Simpson rule. The parameter $n$ still means that we should be evaluating the function at equispaced points $x_0 = a < x_1 < x_2 < \dots < x_n=b$

In [3]:
function simpson(f::Function, a::Real, b::Real, n::Integer=100)
    sum = 0.
    h = (b-a)/n
    sum += (f(a)+f(b))*h/3
    for i in 1:n-1
        xi = h*i + a
        if mod(i,2)==1 
            sum += 4*h*f(xi)/3
        else
            sum += 2*h*f(xi)/3
        end
    end
    return sum
end

simpson (generic function with 2 methods)

The following computation should give you an exact value up to round-off errors. Why?

In [4]:
simpson(x->x^2,0,1)

0.33333333333333337

The following function implement Gaussian curvature with two intermediate points. Note that unline the previous cases, we end up evaluating the function $f$ in more than $n+1$ points.

In [5]:
function gaussian2(f::Function, a::Real, b::Real, n::Integer=100)
    sum = 0.
    h = (b-a)/n
    root = sqrt(1/3)
    for i in 0:n-1
        x0 = h*i + a
        x1 = h*(i+1) + a
        
        y1 = (x0+x1)/2 + root*(x1-x0)/2
        y2 = (x0+x1)/2 - root*(x1-x0)/2
        sum += (f(y1)+f(y2))*h/2
    end
    return sum
end    

gaussian2 (generic function with 2 methods)

In [6]:
# The following computation should also be exact
gaussian2(x->x^2,0,1)

0.3333333333333333

You should now implement gaussian3 that uses three intermediate points instead of two. You can use [Wikipedia](https://en.wikipedia.org/wiki/Gaussian_quadrature) to find the roots of the orthogonal polynomial and the corresponding weights.

In [7]:
function gaussian3(f::Function, a::Real, b::Real, n::Integer=100)
    sum = 0.
    h = (b-a)/n
    root = sqrt(3/5)
    α1 = 5 / 18
    α2 = 5 / 18
    α3 = 4/9
    for i in 0:n-1
        x0 = h*i + a
        x1 = h*(i+1) + a
        
        y1 = (x0+x1)/2 + root*(x0-x1)/2
        y2 = (x0+x1)/2 - root*(x0-x1)/2
        y3 = (x0+x1)/2
        sum += (α1*f(y1)+α1*f(y2)+α3*f(y3))*h
    end
    return sum
end    

gaussian3 (generic function with 2 methods)

In [8]:
# The following computation should also be exact
gaussian3(x->x^2,0,1)

0.3333333333333333

## Computing $\pi$

There are several smart ways to compute the number $\pi$ with good accuracy. We are going to do something that is not so smart. Let us approximate $\pi$ by computing the area of a circle directly.

$$ \pi = 2 \int_{-1}^1 \sqrt{1-x^2} \, dx . $$

Let us test our algorithms to see what we get.

In [9]:
println("Exact value: ", Float64(π))
circle(x) = 2*sqrt(1-x^2)
println("Approximation by trapezoid rule:", trapezoid(circle,-1,1))
println("Approximation by Simpson rule:", simpson(circle,-1,1))
println("Approximation by Gaussian quadrature with two points:", gaussian2(circle,-1,1))
println("Approximation by Gaussian quadrature with three points:", gaussian3(circle,-1,1))

Exact value: 3.141592653589793
Approximation by trapezoid rule:3.138268511098501
Approximation by Simpson rule:3.140292577830334
Approximation by Gaussian quadrature with two points:3.141709421681971
Approximation by Gaussian quadrature with three points:3.1416328913577884


The number above are aguably unfair. The trapezoid and simpson rule (if you implement them like I did) evaluate the function in $n+1$ points. The gaussian2 evaluates the function in $2n$ points, and the gaussian3 evaluates the function in $3n$ point. To make the competition fair, let us adapt the value of $n$ accordingly.

In [10]:
println("Exact value: ", Float64(π))
circle(x) = 2*sqrt(1-x^2)
println("Approximation by trapezoid rule: ", trapezoid(circle,-1,1,600))
println("Approximation by Simpson rule: ", simpson(circle,-1,1,600))
println("Approximation by Gaussian quadrature with two points: ", gaussian2(circle,-1,1,300))
println("Approximation by Gaussian quadrature with three points: ", gaussian3(circle,-1,1,200))

Exact value: 3.141592653589793
Approximation by trapezoid rule: 3.141366358851329
Approximation by Simpson rule: 3.141504258208093
Approximation by Gaussian quadrature with two points: 3.141615112264936
Approximation by Gaussian quadrature with three points: 3.141606877131482


I am getting approximately four digits of accuracy with the four algorithms. This is not a very significant difference between the four algorithms, and it is maybe less that we would have expected. **Can you explain why the accuracy is so low in this example?**

Let us now compute a different function as a way to approximate $\pi$. We will use the following identity.
$$ \pi = \int_{-1}^1 \frac 2 {1+x^2} \, dx.$$

In [11]:
println("Exact value: ", Float64(π))
g(x) = 2/(1+x^2)
println("Approximation by trapezoid rule: ", trapezoid(g,-1,1,60))
println("Approximation by Simpson rule: ", simpson(g,-1,1,60))
println("Approximation by Gaussian quadrature with two points: ", gaussian2(g,-1,1,30))
println("Approximation by Gaussian quadrature with three points: ", gaussian3(g,-1,1,20))

Exact value: 3.141592653589793
Approximation by trapezoid rule: 3.1414074684073294
Approximation by Simpson rule: 3.1415926535353593
Approximation by Gaussian quadrature with two points: 3.141592653628502
Approximation by Gaussian quadrature with three points: 3.141592653560033


In this case, the Simpson rule, and the two Gaussian quadrature rules, give us a much better approximation than the trapezoid rule. The approximations are accurate to several digits even though we used very few points.

The function `gaussian3` has a higher degree of accuracy in theory than `simpson` or `gaussian2`. However, to really apreciate its benefit, we would have to be integrating with a larger number of points. In practice, we get a nearly exact solution with any of these three methods if we use more than 150 points. So, the benefit of `gaussian3` is harder to observe.